# Sprint 自然言語処理入門
## 1.このSprintについて
### Sprintの目的
自然言語処理の一連の流れを学ぶ
自然言語のベクトル化の方法を学ぶ
### どのように学ぶか
自然言語処理定番のデータセットを用いて、一連の流れを見ていきます。

## 2.自然言語のベクトル化
自然言語処理（NLP, Natural Language Processing） とは人間が普段使っている 自然言語 をコンピュータに処理させる技術のことです。  
ここではその中でも、機械学習の入力として自然言語を用いることを考えていきます。  

多くの機械学習手法は 数値データ（量的変数） の入力を前提にしていますので、自然言語の テキストデータ を数値データに変換する必要があります。  
これを 自然言語のベクトル化 と呼びます。ベクトル化の際にテキストデータの特徴をうまく捉えられるよう、様々な手法が考えられてきていますので、
このSprintではそれらを学びます。

3.IMDB映画レビューデータセットの準備
IMDB映画レビューデータセットを準備します。

ダウンロード
次のwgetコマンドによってダウンロードします。

In [1]:
# IMDBをカレントフォルダにダウンロード
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
!cat aclImdb/README

--2019-09-27 17:03:49--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
ai.stanford.edu (ai.stanford.edu) をDNSに問いあわせています... 171.64.68.10
ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 84125825 (80M) [application/x-gzip]
`aclImdb_v1.tar.gz.5' に保存中

aclImdb_v1.tar.gz.5 100%[===================>]  80.23M  8.46MB/s 時間 11s        

2019-09-27 17:04:00 (7.39 MB/s) - `aclImdb_v1.tar.gz.5' へ保存完了 [84125825/84125825]

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000

以下のサイトで公開されているデータセットです。

[リンクの練習](リンクのアドレス "リンクのタイトル")
[Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/ "Sentiment Analysis")

`読み込み`  
scikit-learnのload_filesを用いて読み込みます。

[sklearn.datasets.load_files — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html "sklearn.datasets.load_files — scikit-learn 0.21.3 documentation")

In [2]:
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target

# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos']


### このデータセットについて
中身を見てみると、英語の文章が入っていることが分かります。


In [3]:
print("x : {}".format(x_train[0]))

x : Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.


IMDBはInternet Movie Databaseの略で、映画のデータベースサイトです。

[Ratings and Reviews for New Movies and TV Shows - IMDb](https://www.imdb.com "Ratings and Reviews for New Movies and TV Shows - IMDb")

このサイトではユーザーが映画に対して1から10点の評価とコメントを投稿することができます。そのデータベースから訓練用データは25000件、テスト用データは25000件のデータセットを作成しています。

4点以下を否定的、7点以下を肯定的なレビューとして2値のラベル付けしており、これにより感情の分類を行います。5,6点の中立的なレビューはデータセットに含んでいません。また、ラベルは訓練用・テスト用それぞれで均一に入っています。詳細はダウンロードしたREADMEを確認してください。  
  
## 4.古典的な手法
古典的ながら現在でも強力な手法であるBoWとTF-IDFを見ていきます。

## 5.BoW
単純ながら効果的な方法として BoW (Bag of Words) があります。  
これは、サンプルごとに単語などの 登場回数 を数えたものをベクトルとする方法です。単語をカテゴリとして捉え one-hot表現 していることになります。

### 例
例として、IMDBデータセットからある3文の最初の5単語を抜き出したものを用意しました。

In [4]:
mini_dataset = \
  ["This movie is very good.",
  "This film is a good",
  "Very bad. Very, very bad."]

この3文にBoWを適用させてみます。scikit-learnのCountVectorizerを利用します。

[sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html "sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation")

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()

# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)
# 実行すると次のような表が得られます。

,a,bad,film,good,is,movie,this,very
0,0,0,0,1,1,1,1,1
1,1,0,1,1,1,0,1,0
2,0,2,0,0,0,0,0,3


例にあげた3文の中で登場する8種類の単語が列名になり、0,1,2番目のサンプルでそれらが何回登場しているかを示しています。2番目のサンプル「Very bad. Very, very bad.」ではbadが2回、veryが3回登場しています。列名になっている言葉はデータセットが持つ 語彙 と呼びます。

テキストはBoWにより各サンプルが語彙数の次元を持つ特徴量となり、機械学習モデルへ入力できるようになります。この時使用したテキスト全体のことを コーパス と呼びます。語彙はコーパスに含まれる言葉よって決まり、それを特徴量としてモデルの学習を行います。そのため、テスト用データではじめて登場する語彙はベクトル化される際に無視されます。

### 前処理
CountVectorizerクラスでは大文字は小文字に揃えるという 前処理 が自動的に行われています。こういった前処理は自然言語処理において大切で、不要な記号などの消去（テキストクリーニング）や表記揺れの統一といったことを別途行うことが一般的です。

語形が「see」「saw」「seen」のように変化する単語に対して語幹に揃える ステミング と呼ばれる処理を行うこともあります。

### トークン
BoWは厳密には単語を数えているのではなく、 トークン（token） として定めた固まりを数えます。

何をトークンとするかはCountVectorizerでは引数token_patternで 正規表現 の記法により指定されます。デフォルトはr'(?u)\b\w\w+\b'ですが、上の例ではr'(?u)\b\w+\b'としています。

デフォルトでは空白・句読点・スラッシュなどに囲まれた2文字以上の文字を1つのトークンとして抜き出すようになっているため、「a」や「I」などがカウントされません。英語では1文字の単語は文章の特徴をあまり表さないため、除外されることもあります。しかし、上の例では1文字の単語もトークンとして抜き出すように引数を指定しています。

### 《正規表現》

正規表現は前処理の際にも活用しますが、ここでは詳細は扱いません。Pythonではreモジュールによって正規表現操作ができます。

[re — 正規表現操作](https://docs.python.org/ja/3/library/re.html "re — 正規表現操作")

正規表現を利用する際はリアルタイムで結果を確認できる以下のようなサービスが便利です。

[Online regex tester and debugger: PHP, PCRE, Python, Golang and JavaScript](https://regex101.com "Online regex tester and debugger: PHP, PCRE, Python, Golang and JavaScript")

### 形態素解析
英語などの多くの言語では空白という分かりやすい基準でトークン化が行えますが、日本語ではそれが行えません。

日本語では名詞や助詞、動詞のように異なる 品詞 で分けられる単位で 分かち書き することになります。例えば「私はプログラミングを学びます」という日本語の文は「私/は/プログラミング/を/学び/ます」という風になります。

これには MeCab や Janome のような形態素解析ツールを用います。Pythonから利用することも可能です。MeCabをウェブ上で簡単に利用できる  
[Web茶まめ](http://chamame.ninjal.ac.jp/ "Web茶まめ")
というサービスも国立国語研究所が提供しています。

自然言語では新しい言葉も日々生まれますので、それにどれだけ対応できるかも大切です。MeCab用の毎週更新される辞書として mecab-ipadic-NEologd がオープンソースで存在しています。

[mecab-ipadic-neologd/README.ja.md at master · neologd/mecab-ipadic-neologd](https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md "mecab-ipadic-neologd/README.ja.md at master · neologd/mecab-ipadic-neologd")

n-gram
上のBoWの例では1つの単語（トークン）毎の登場回数を数えましたが、これでは語順は全く考慮されていません。

考慮するために、隣あう単語同士をまとめて扱う n-gram という考え方を適用することがあります。2つの単語をまとめる場合は 2-gram (bigram) と呼び、次のようになります。

In [6]:
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a good,bad very,film is,is a,is very,movie is,this film,this movie,very bad,very good,very very
0,0,0,0,0,1,1,0,1,0,1,0
1,1,0,1,1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,2,0,1


2-gramにより「very good」と「very bad」が区別して数えられています。

単語をまとめない場合は 1-gram (unigram) と呼びます。
3つまとめる3-gram(trigram)など任意の数を考えることができます。1-gramと2-gramを組み合わせてBoWを行うといったこともあります。

## 【問題1】BoWのスクラッチ実装
以下の3文のBoWを求められるプログラムをscikit-learnを使わずに作成してください。1-gramと2-gramで計算してください。

```
This movie is SOOOO funny!!!
What a movie! I never
best movie ever!!!!! this movie
```

In [7]:
mini_dataset2 = \
  ["This movie is SOOOO funny!!!",
  "What a movie! I never",
  "best movie ever!!!!! this movie"]

In [8]:
# 動作確認
# 1-gram
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset2)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0,0,0,1,0,1,1,0,1,1,0
1,1,0,0,0,1,0,1,1,0,0,1
2,0,1,1,0,0,0,2,0,0,1,0


In [9]:
# 動作確認
# 2-gram
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset2)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a movie,best movie,ever this,i never,is soooo,movie ever,movie i,movie is,soooo funny,this movie,what a
0,0,0,0,0,1,0,0,1,1,1,0
1,1,0,0,1,0,0,1,0,0,0,1
2,0,1,1,0,0,1,0,0,0,1,0


In [10]:
# 1-gram
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(x_train)).toarray()
df1 = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df1.head()

,0,00,000,0000000000001,00001,00015,000s,001,003830,006,...,état,étc,évery,êxtase,ís,ísnt,ø,østbye,über,üvegtigris
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 2-gram
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(x_train)).toarray()
df2 = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df2.head()

,0 0,0 00001,0 02,0 1,0 10,0 2,0 3,0 48,0 5,0 6,...,évery scene,êxtase ecstasy,ís brilliant,ísnt entertaining,østbye is,über alles,über annoying,über berlin,über spy,üvegtigris so
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# スクラッチしようとしたけど断念
# import numpy as np

# docs = np.array([mini_dataset2])
# print(docs)

text = '"This movie is SOOOO funny!!!",  "What a movie! I never",  "best movie ever!!!!! this movie"'
text = text.split(' ')
print(text)

# print(text.split(' '))
# print(mini_dataset2.split(' '))
# print(docs.split(' '))

# 文字列をスペースごとに区切る
# テキストを全て見て、それをユニークなカラムとする
#ベクトルにする、配列にする


['"This', 'movie', 'is', 'SOOOO', 'funny!!!",', '', '"What', 'a', 'movie!', 'I', 'never",', '', '"best', 'movie', 'ever!!!!!', 'this', 'movie"']


### 6.TF-IDF
BoWの発展的手法として TF-IDF もよく使われます。これは Term Frequency (TF) と Inverse Document Frequency (IDF) という2つの指標の組み合わせです。
IDF
IDFはそのトークンがデータセット内で珍しいほど値が大きくなる指標です。

サンプル数 
N をIMDB映画レビューデータセットの訓練用データに合わせ25000として、トークンが出現するサンプル数 df(t) を変化させたグラフを確認してみると、
次のようになります。

In [13]:
import numpy as np
import matplotlib.pyplot as plt
n_samples = 25000
idf = np.log(n_samples/np.arange(1,n_samples))
plt.title("IDF")
plt.xlabel("df(t)")
plt.ylabel("IDF")
plt.plot(idf)
plt.show()

TF-IDFではこの数を出現回数に掛け合わせるので、珍しいトークンの登場に重み付けを行なっていることになります。

### ストップワード
あまりにも頻繁に登場するトークンは、値を小さくするだけでなく、取り除くという前処理を加えることもあります。
取り除くもののことを ストップワード と呼びます。既存のストップワード一覧を利用したり、しきい値によって求めたりします。

scikit-learnのCountVectorizerでは引数stop_wordsにリストで指定することで処理を行なってくれます。

In [14]:
# 動作確認用
vectorizer = CountVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a,bad,film,good,movie,this,very
0,0,0,0,1,1,1,1
1,1,0,1,1,0,1,0
2,0,2,0,0,0,0,3


In [15]:
# 実データ
vectorizer = CountVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b')
bow_train = (vectorizer.fit_transform(x_train)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df.head()

,0,00,000,0000000000001,00001,00015,000s,001,003830,006,...,état,étc,évery,êxtase,ís,ísnt,ø,østbye,über,üvegtigris
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


代表的な既存のストップワード一覧としては、 *NLTK*  という自然言語処理のライブラリのものがあげられます。
あるデータセットにおいては特別重要な意味を持つ単語が一覧に含まれている可能性もあるため、使用する際は中身を確認することが望ましいです。

In [16]:
# はじめて使う場合はストップワードをダウンロード
import nltk
stop_words = nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print("stop word : {}".format(stop_words)) # 'i', 'me', 'my', ...

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tominagashuuji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
stop word : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'whe

逆に、登場回数が特に少ないトークンも取り除くことが多いです。
全てのトークンを用いるとベクトルの次元数が著しく大きくなってしまい計算コストが高まるためです。

scikit-learnのCountVectorizerでは引数max_featuresに最大の語彙数を指定することで処理を行なってくれます。
以下の例では出現数が多い順に5個でベクトル化しています。

In [17]:
# 動作確認
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features = 5)
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df.head()

,bad,good,is,this,very
0,0,1,1,1,1
1,0,1,1,1,0
2,2,0,0,0,3


In [18]:
# 実データ
vectorizer = CountVectorizer(stop_words=stop_words, token_pattern=r'\b\w+\b', max_features = 5000)
bow_train = (vectorizer.fit_transform(x_train)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df.head()

,0,00,000,1,10,100,11,12,13,13th,...,york,young,younger,youth,z,zero,zizek,zombie,zombies,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 【問題2】TF-IDFの計算
IMDB映画レビューデータセットをTF-IDFによりベクトル化してください。
NLTKのストップワードを利用し、最大の語彙数は5000程度に設定してください。テキストクリーニングやステミングなどの前処理はこの問題では要求しません。

TF-IDFの計算にはscikit-learnの以下のどちらかのクラスを使用してください。

In [19]:
# テスト動作確認
# NLTKのストップワードを利用し、最大の語彙数は5000程度に設定する。
stop_words = stopwords.words('english')

# vectorizer = CountVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b')
vectorizer = CountVectorizer(stop_words=stop_words, token_pattern=r'\b\w+\b', max_features = 5000)
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)


,bad,film,good,movie
0,0,0,1,1
1,0,1,1,0
2,2,0,0,0


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# テスト動作確認
corpus = [
'This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df)
print(vectorizer.get_feature_names())
print(X.shape)
print(type(X))
type(X)
print(type(corpus))
print(type(df))
print(type(df.values))

['bad', 'film', 'good', 'movie']
(4, 4)
<class 'scipy.sparse.csr.csr_matrix'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [22]:
# 実データ
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 1), token_pattern=r'\b\w+\b', max_features = 5000)
bow_train = (vectorizer.fit_transform(x_train)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
df.head()

,0,00,000,1,10,100,11,12,13,13th,...,york,young,younger,youth,z,zero,zizek,zombie,zombies,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# 実データ、ベクトル化されているはずなんだがどうやってそのベクトル情報を見れるのだろうか？？
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1), token_pattern=r'\b\w+\b', max_features = 5000)
# X = vectorizer.fit_transform(df)
X = vectorizer.fit_transform(x_train[0:1])
print(vectorizer.get_feature_names())
# print(X.shape)
# print(vectorizer.get_params())
X.toarray()

['actions', 'actors', 'almost', 'anything', 'better', 'beyond', 'bizarre', 'boys', 'br', 'captures', 'civility', 'classmates', 'commit', 'common', 'coupled', 'day', 'decided', 'define', 'destruction', 'elephant', 'even', 'explaining', 'far', 'film', 'filmmaker', 'flawed', 'gets', 'given', 'honest', 'honesty', 'humans', 'leads', 'likely', 'men', 'mode', 'money', 'motives', 'movie', 'murderers', 'must', 'mutual', 'order', 'perfect', 'product', 'rationalistic', 'remarkable', 'see', 'skin', 'slaughtering', 'suicide', 'terms', 'terrible', 'think', 'time', 'two', 'via', 'withdraw', 'world', 'would', 'young', 'zero']


array([[0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.1898316, 0.0949158,
        0.0949158, 0.0949158, 0.3796632, 0.0949158, 0.0949158, 0.0949158,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.1898316, 0.1898316,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.1898316, 0.0949158,
        0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158, 0.0949158,
        0.0949158, 0.1898316, 0.1898316, 0.0949158, 0.1898316, 0.0949158,
        0.2847474, 0.1898316, 0.0949158, 0.0949158, 0.0949158, 0.1898316,
        0.0949158]])

## 【問題3】TF-IDFを用いた学習
問題2で求めたベクトルを用いてIMDB映画レビューデータセットの学習・推定を行なってください。  
モデルは2値分類が行える任意のものを利用してください。

ここでは精度の高さは求めませんが、最大の語彙数やストップワード、n-gramの数を変化させて影響を検証してみてください。

In [41]:
from sklearn.preprocessing import StandardScaler # 標準化

from sklearn.metrics import accuracy_score # 正解率
from sklearn.metrics import precision_score # 適合率
from sklearn.metrics import recall_score # 再現率
from sklearn.metrics import f1_score # F値
from sklearn.metrics import confusion_matrix # 混合行列

from sklearn.linear_model import LogisticRegression # ロジスティック回帰

In [42]:
# # 標準化
# scaler = StandardScaler() # 標準化用のライブラリーをインスタンス化
# x_train_fit = scaler.fit(x_train) # 訓練用データで fit をする。(スカラーデータに変更？)別の変数に格納しない。
# x_test_fit = scaler.fit(x_test) # 訓練用データで fit をする。(スカラーデータに変更？)別の変数に格納しない。

# x_train_transform = scaler.transform(x_train)
# x_test_transform = scaler.transform(x_test)

In [43]:
# ロジクスティック回帰
# logistic regression model 
logreg = LogisticRegression()
logreg = logreg.fit(df, y_train)

y_pred_logreg = logreg.predict(df)

cnf_matrix_logreg = confusion_matrix(y_test, y_pred_logreg)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
precision_logreg = precision_score(y_test, y_pred_logreg)
recall_logreg = recall_score(y_test, y_pred_logreg)
f1_logreg = f1_score(y_test, y_pred_logreg)

In [47]:
def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica']):
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2

    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = model.predict(mesh).reshape(mesh_f0.shape)

    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()

In [48]:
decision_region(df, y_train, logreg) # ロジスティック回帰

TypeError: '(slice(None, None, None), 0)' is an invalid key